In [1]:
import json

# Ingrese sus claves / secretos como cadenas en los siguientes campos
credentials = {}
credentials['CONSUMER_KEY'] = 'kQXm8Zqg0W6BMi3KlDcx3oApA'
credentials['CONSUMER_SECRET'] = 'YROLVjxp9yPS0LrRLoQ3VeQP3ZiwTt9etHPXXg15EqX8moeXBK'
credentials['ACCESS_TOKEN'] = '1520180221-whZMAPKk05wDNEYNHY37znL2f40MfiwWeBadGmB'
credentials['ACCESS_SECRET'] = 'NH6ROjDOIpmoSYUGv9NDkR7b43zp6XEFyczO0HSaK5cUU'

# Guardar el objeto de credenciales en el archivo
with open("twitter_credentials.json", "w") as file:
    json.dump(credentials, file)

In [2]:
# Importar la clase Twython
from twython import Twython
import json

# Cargar credenciales del archivo json
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)
    
# Instanciar un objeto
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# Crea tu consulta
query = {'q': 'Microsoft',
        'result_type': 'popular',
        'count': 10,
        'lang': 'en',
        }

In [3]:
import pandas as pd

# Buscar tweets
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}
for status in python_tweets.search(**query)['statuses']:
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])

# Estructura de datos en un DataFrame de pandas para una manipulación más fácil
df = pd.DataFrame(dict_)
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(5)

,user,date,text,favorite_count
0,JonErlichman,Sat Feb 08 21:31:16 +0000 2020,Age when they co-founded these tech companies:...,836
1,IGN,Sun Feb 09 11:01:00 +0000 2020,ICYMI: Phil Spencer says he has a ton of respe...,820
2,verge,Sun Feb 09 06:54:09 +0000 2020,You can now make sick beats in Microsoft Excel...,527
6,IGN,Sat Feb 08 07:31:00 +0000 2020,Nintendo’s Shigeru Miyamoto says that spreadin...,396
4,eurogamer,Sat Feb 08 18:08:06 +0000 2020,Someone's leaked Microsoft Flight Simulator ga...,334


Ahora se va a proceder a realizar un procedimiento para recopilar 
información de froma seuencial con el API de Streaming de 
twitter, en esta aparte vamos a usatr TwythonStreaming.

Vamos a necesitar realizar una clase MyStreamer que herede 
TwythonStreamer y luego anular los metodos on_success y
on_error, de la siguiente forma:

Se debe de tomar en cuenta que el metodo On_Success se llama automaticamente cuando twitter nos envia los datos, mientras que on_errorvada vez que existe un error en el API por posibles restricciones, el metodo que se va agregar en el codigos  "save_to_csv" es una forma que vamos autilizar par almacenar tweets.

In [4]:
from twython import TwythonStreamer
import csv

# Filtrar datos no deseados
def process_tweet(tweet):
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d
    
    
# Crear una clase que herede TwythonStreamer
class MyStreamer(TwythonStreamer):     

 # Datos recibidos
    def on_success(self, data):

        # Solo recopila tweets en español
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_csv(tweet_data)

   # Problema con la API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()
        
    # Guardar cada tweet en un archivo csv
    def save_to_csv(self, tweet):
        with open(r'saved_tweets.csv', 'a',encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))

Ahora se va a proceder a crear uan instancia  del objeto MyStreamer 
con la credenciales iniciales, donde se van agregar filtros
para obtener solos los tweets que más son de interes en est caso o vamos 
a realizar con la palabra "Microsoft"

In [5]:
# Instanciar desde nuestra clase de transmisión
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'], 
                    creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
# Iniciar la transmisión
stream.statuses.filter(track='Microsoft')

KeyboardInterrupt: 

Con el codigo anterio se recolecto mucha información,
ahora vamos a proceder a analizar la información
    

In [6]:
import pandas as pd
tweets = pd.read_csv("saved_tweets.csv")
tweets.head()

,hashtags,text,user,location
0,[],@kidsmoove You must be trolling? No used games...,HardcoreMetal89,"Melbourne, Australia"
1,[],@mread531 @TheRealCBlakely @Microsoft I can't ...,soundertillidie,University Place
2,[],RT @migueldeicaza: AMD contributes a backend t...,hasssaaannn,"Stockholm, Sweden"
3,[],@staryeyes113 @CStomp10 @Alexmai10086759 @Post...,AgitatingK,"Lautoka City, Fiji."
4,['Earnings'],RT @danielnewmanUV: We are back baby! #Earning...,SaiKrishanKumar,"Valencia, Spain"


In [7]:
from collections import Counter
import ast

tweets = pd.read_csv("saved_tweets.csv")
tweets



,hashtags,text,user,location
0,[],@kidsmoove You must be trolling? No used games...,HardcoreMetal89,"Melbourne, Australia"
1,[],@mread531 @TheRealCBlakely @Microsoft I can't ...,soundertillidie,University Place
2,[],RT @migueldeicaza: AMD contributes a backend t...,hasssaaannn,"Stockholm, Sweden"
3,[],@staryeyes113 @CStomp10 @Alexmai10086759 @Post...,AgitatingK,"Lautoka City, Fiji."
4,['Earnings'],RT @danielnewmanUV: We are back baby! #Earning...,SaiKrishanKumar,"Valencia, Spain"
...,...,...,...,...
118,[],Microsoft Excel is where i get mine from.,temmyd,NaN
119,[],RT @NthAsia: The Gucci belt they don't tell yo...,Fa1ade,NaN
120,"['github', 'githubaction', 'oss', 'microsoft']",RT @sudip_post: Just tried my hands on github-...,mayur_shingote,"Pune, India"
121,[],RT @evankirstel: Bill Gates becomes first to b...,ianO6Bon,🌍


A continuación, podemos usar la ubicación del usuario para responder: ¿qué áreas del mundo tuitean más sobre "python"? Para este paso, utilizaremos el método de geocodificación de la biblioteca de geopy que devuelve las coordenadas de una ubicación de entrada dada. Para visualizar un mapa de calor mundial de tweets, usaremos la biblioteca gmplot. Un recordatorio: nuestros pequeños datos no son un representante real del mundo.

In [8]:
from geopy.geocoders import Nominatim
import gmplot

geolocator = Nominatim(user_agent="BD-CR-APP")

# Revisa todos los tweets y agrega ubicaciones al diccionario de "coordenadas"
coordinates = {'latitude': [], 'longitude': []}
for count, user_loc in enumerate(tweets.location):
    try:
        location = geolocator.geocode(user_loc)
        
      # Si se encuentran coordenadas para la ubicación
        if location:
            coordinates['latitude'].append(location.latitude)
            coordinates['longitude'].append(location.longitude)
            
   # Si hay demasiadas solicitudes de conexión
    except:
        pass
    
# Instanciar y centrar un objeto GoogleMapPlotter para mostrar nuestro mapa
gmap = gmplot.GoogleMapPlotter(30, 0, 3)

# Insertar puntos en el mapa pasando una lista de latitudes y longitudes
gmap.heatmap(coordinates['latitude'], coordinates['longitude'], radius=20)

# Guardar el mapa en un archivo html
gmap.draw("C:\\Users\\Admin-TI\\Desktop\\python_heatmap.html")

El código anterior produjo el mapa de calor en la siguiente figura, que muestra una mayor actividad en los tweets "Microsoft" en Estados Unidos, Alemania y francia. Una desventaja del enfoque descrito es que no hicimos ninguna limpieza de datos; resultaron ser muchos tweets generados por máquina que provienen de una sola ubicación, o múltiples ubicaciones que producen un mismo tweet. Por supuesto, estas muestras deben descartarse, para obtener una imagen más realista de la distribución geográfica de los humanos tuiteando "Microsoft". Una segunda mejora sería simplemente recopilar más datos durante períodos más largos e ininterrumpidos.

Usar pyspark SQL para consultar la información

In [9]:
#importamos la bibliotecas

from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('data_processing').getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.types import *
import findspark
from datetime import datetime
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType
findspark.init('C:\Spark\spark-2.4.4-bin-hadoop2.7')

In [10]:
#Cargamos el archivo de Excel
df = spark.read.csv('saved_tweets.csv',header=True, inferSchema = True)

In [11]:
#Hacemos un Summary de la información
df.summary().show()

+-------+---------------+--------------------+--------------------+--------------------+
|summary|       hashtags|                text|                user|            location|
+-------+---------------+--------------------+--------------------+--------------------+
|  count|            188|                 158|                 118|                  59|
|   mean|           null|                 0.0|                null|                null|
| stddev|           null|                 NaN|                null|                null|
|    min| ""This digital| @Microsoft @msex...| learning solution…"|353 Island Dr Mad...|
|    25%|           null|                 0.0|                null|                null|
|    50%|           null|                 0.0|                null|                null|
|    75%|           null|                 0.0|                null|                null|
|    max|via @GoogleNews|       zouriegraphix|          yorguenaut|                  🌍|
+-------+-------------

In [12]:
#Solo consultamos los primeros 25 por texto y usuario
df.select(['text','user']).show(25)

+--------------------+--------------------+
|                text|                user|
+--------------------+--------------------+
|@kidsmoove You mu...|     HardcoreMetal89|
|@mread531 @TheRea...|     soundertillidie|
|RT @migueldeicaza...|         hasssaaannn|
|@staryeyes113 @CS...|          AgitatingK|
|RT @danielnewmanU...|     SaiKrishanKumar|
|"RT @mkashman: Ad...|                null|
|        customizable| learning solution…"|
|RT @BTSx50States:...|                null|
|                null|                null|
|     janetvincent721|BTS ×ARMY🌏𝐁𝐋𝐀...|
|Microsoft offers ...|      CyberScoopNews|
|RT @BTSx50States:...|                null|
|                null|                null|
|        BTSkittylove|Redmond, WA and a...|
|From chatbots to ...|           NCSoffice|
|A great opportuni...|      CristinGoodwin|
|If this statement...|            LORD97A1|
|@Q4thWall1 The ra...|     JajaPeter777777|
|              #NAME?|               C7ron|
|Tested Minecraft ...|     EastRiver

Usar SQL QUERY donde vamos a buscar unicamente los tweets que contengan la palabra "Microsoft" en el texto

In [13]:
#como primera parte vamos a consultar el total de Tweets 
df.createOrReplaceTempView("df")
sqlDF = spark.sql("SELECT count(text) FROM df ")
sqlDF.show()

+-----------+
|count(text)|
+-----------+
|        158|
+-----------+



Como resultado nos arroja 128 en total del archivo de Excel, ahora vamos a contar cuantos tienen la palabra "Microsoft" en el texto con el fin de validar más la información con el siguiente Query de SQL

In [14]:
sqlDF = spark.sql("SELECT count(text) FROM df where text like '%Microsoft%' ")
sqlDF.show()

+-----------+
|count(text)|
+-----------+
|         62|
+-----------+



Como resultado obtenemos que solo 51 de los 128 son los qe hacen mencion en su texto de la palabra "Microsoft" y en el siguiente caso solo vamos a motrar los registros con ese contenido: 

In [15]:
sqlDF = spark.sql("SELECT text, user FROM df where text like '%Microsoft%' ")
sqlDF.show(62)

+--------------------+---------------+
|                text|           user|
+--------------------+---------------+
|@mread531 @TheRea...|soundertillidie|
|RT @migueldeicaza...|    hasssaaannn|
|@staryeyes113 @CS...|     AgitatingK|
|RT @danielnewmanU...|SaiKrishanKumar|
|"RT @mkashman: Ad...|           null|
|Microsoft offers ...| CyberScoopNews|
|A great opportuni...| CristinGoodwin|
|If this statement...|       LORD97A1|
|Tested Minecraft ...|EastRiverOnline|
|RT @SwiftOnSecuri...|           null|
|RT @jarrytania: M...|   KaurRajpriya|
|RT @SwiftOnSecuri...|           null|
| @Microsoft @msex...|   BhuvanPasham|
|Remember to take ...|        Pixar86|
|"RT @zahid_XE: Az...| dotnet_careers|
|@HiddenXperia bec...|   Jinx87880518|
|RT @SwiftOnSecuri...|           null|
|I just entered to...| Notacontestguy|
|RT @jeremyphoward...|     NgJiuanJye|
|RT @SwiftOnSecuri...|           null|
|Big Data in Power...|  shimla_online|
|Sigh. Microsoft, ...|   r_keith_hill|
|RT @ryan_levick: ...|   